In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import json
#from google.colab import drive
import torch
from tqdm import tqdm
import os

In [2]:
os.chdir('/home2/skylee/GP')

In [3]:
# GPU 할당 변경하기
GPU_NUM = 1 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

print ('Current cuda device:', torch.cuda.current_device()) # check

Current cuda device: 1


In [4]:
#pip install -U sentence-transformers

In [5]:
pip install --no-deps sentence-transformers tqdm numpy scikit-learn scipy nltk transformers tokenizers requests


  Using cached https://files.pythonhosted.org/packages/72/8a/34efae5cf9924328a8f34eeb2fdaae14c011462d9f0e3fcded48e1266d1c/tqdm-4.60.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f5/ef/bcd79e8d59250d6e8478eb1290dc6e05be42b3be8a86e3954146adbc171a/scikit_learn-0.24.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c8/89/63171228d5ced148f5ced50305c89e8576ffc695a90b58fe5bb602b910c2/scipy-1.5.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5e/37/9532ddd4b1bbb619333d5708aaad9bf1742f051a664c3c6fa6632a105fd8/nltk-3.6.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d8/b2/57495b5309f09fa501866e225c84532d1fd89536ea62406b2181933fb418/transformers-4.5.1-py3-none-any.whl
  Using cached https://file

In [6]:
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Epinion

In [7]:
r_epi_82=pd.read_table('review_epi_82.txt',header=None)
r_epi_91=pd.read_table('review_epi_91.txt',header=None)

In [8]:
r_epi_82.columns=['review']
for i in range(len(r_epi_82)):
  r_epi_82.loc[i,'review']=r_epi_82.loc[i,'review'][2:]
r_epi_82.head()

,review
0,i got this printer from minolta as a warranty ...
1,bought this at best buy for 6999 9999 30 mail ...
2,bought this printer at costco for 99 after reb...
3,well when i graduated from college it didnt ta...
4,i have a firebird and it is a very dependable ...


In [9]:
r_epi_91.columns=['review']
for i in range(len(r_epi_91)):
  r_epi_91.loc[i,'review']=r_epi_91.loc[i,'review'][2:]
r_epi_91.head()

,review
0,i got this printer from minolta as a warranty ...
1,bought this at best buy for 6999 9999 30 mail ...
2,bought this printer at costco for 99 after reb...
3,just recently my sony dej001 walkman died on m...
4,i was searching around walmart for a new boomb...


In [ ]:
#Epinion82
m_epi_82=pd.DataFrame(index=range(len(r_epi_82)),columns=range(20))
score_epi_82=pd.DataFrame(index=range(len(r_epi_82)),columns=range(20))

corpus=r_epi_82['review']
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
queries=corpus

top_k=min(21,len(corpus))

for idx,query in enumerate(tqdm(queries)):
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    top_idx=top_results[1].to('cpu').numpy()[1:]
    top_scores=top_results[0].to('cpu').numpy()[1:]

    for t in range(len(top_idx)):
      m_epi_82.loc[idx,t] = top_idx[t]
      scores=[str(top_idx[t]),top_scores[t]]
      score_epi_82.loc[idx,t]=scores

m_epi_82.to_csv('review_idx_epi82.txt',index=True,header=False)
score_epi_82.to_csv('score_epi82.txt',index=True,header=False)

In [ ]:
#Epinion91
m_epi_91=pd.DataFrame(index=range(len(r_epi_91)),columns=range(20))
score_epi_91=pd.DataFrame(index=range(len(r_epi_91)),columns=range(20))

corpus=r_epi_91['review']
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
queries=corpus

top_k=min(21,len(corpus))

for idx,query in enumerate(tqdm(queries)):
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    top_idx=top_results[1].to('cpu').numpy()[1:]
    top_scores=top_results[0].to('cpu').numpy()[1:]

    for t in range(len(top_idx)):
      m_epi_91.loc[idx,t] = top_idx[t]
      scores=[str(top_idx[t]),top_scores[t]]
      score_epi_91.loc[idx,t]=scores

m_epi_91.to_csv(path+'review_idx_epi91.txt',index=True,header=False)
score_epi_91.to_csv(path+'score_epi91.txt',index=True,header=False)


# Librarything

In [ ]:
r_lib_82=pd.read_table('./preprocessing_data/librarything82/review.txt',header=None)

In [7]:
r_lib_82.columns=['review']
for i in range(len(r_lib_82)):
  r_lib_82.loc[i,'review']=r_lib_82.loc[i,'review'][2:]
r_lib_82.head()

,review
0,This a great book for young readers to be intr...
1,Interesting and offputting at the same time W...
2,Possibly would have been better if I had left ...
3,Outstanding the best book in the series Rowlin...
4,Two stories set around Christmas written by No...


In [8]:
#Librarything82
m_lib_82=pd.DataFrame(index=range(len(r_lib_82)),columns=range(20))
score_lib_82=pd.DataFrame(index=range(len(r_lib_82)),columns=range(20))

corpus=r_lib_82['review']
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
queries=corpus

top_k=min(21,len(corpus))

for idx,query in enumerate(tqdm(queries)):
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    top_idx=top_results[1].to('cpu').numpy()[1:]
    top_scores=top_results[0].to('cpu').numpy()[1:]

    for t in range(len(top_idx)):
      m_lib_82.loc[idx,t] = top_idx[t]
      scores=[str(top_idx[t]),top_scores[t]]
      score_lib_82.loc[idx,t]=scores

m_lib_82.to_csv(path+'review_idx_lib82.txt',index=True,header=False)
score_lib_82.to_csv(path+'score_lib82.txt',index=True,header=False)

In [13]:
r_lib_91=pd.read_table(path+'review_lib_91.txt',header=None)

In [14]:
r_lib_91.columns=['review']
for i in range(len(r_lib_91)):
  r_lib_91.loc[i,'review']=r_lib_91.loc[i,'review'][2:]
r_lib_91.head()

,review
0,This a great book for young readers to be intr...
1,Interesting and offputting at the same time W...
2,Possibly would have been better if I had left ...
3,Outstanding the best book in the series Rowlin...
4,Two stories set around Christmas written by No...


In [ ]:
#Librarything91
m_lib_91=pd.DataFrame(index=range(len(r_lib_91)),columns=range(20))
score_lib_91=pd.DataFrame(index=range(len(r_lib_91)),columns=range(20))

corpus=r_lib_91['review']
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
queries=corpus

top_k=min(21,len(corpus))

for idx,query in tqdm(enumerate(queries)):
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    top_idx=top_results[1].to('cpu').numpy()[1:]
    top_scores=top_results[0].to('cpu').numpy()[1:]

    for t in range(len(top_idx)):
      m_lib_91.loc[idx,t] = top_idx[t]
      scores=[str(top_idx[t]),top_scores[t]]
      score_lib_91.loc[idx,t]=scores

m_lib_91.to_csv(path+'review_idx_lib91.txt',index=True,header=False)
score_lib_91.to_csv(path+'score_lib91.txt',index=True,header=False)

168818it [2:33:32, 19.21it/s]